In [1]:
import main

malloc_trim available: True


In [2]:
config_path = 'config_lucerne.json'

In [ ]:
main.main(config_path)

Creating output folders in: /storage/alplakes_test/lucerne_2025/outputs_swirl_01
No SLURM detected. Using config file: = 50 cores
No SLURM memory allocation detected. Using available system memory: 252.0 GB
Auto-calculated memory per worker: 4.54 GB
Dask cluster configured:
  Workers: 50
  Memory per worker: 4535359635B
  Dashboard: http://127.0.0.1:8787/status
iterations: 
[     72     216     360 ... 1050264 1050408 1050552]
Running iteration 0 out of 7296.
Loading input data for iteration [72]... (2025-11-19 10:52:37)
[72]
Preparing parallel tasks... (2025-11-19 10:52:53)

Task chunking summary:
  1 time chunks × 50 depth chunks = 50 total tasks
  Each task processes ~1 pixels (50 pixels total)
  50 workers available → ~1.0 tasks per worker

Computing parallel tasks... (2025-11-19 10:52:53)


  0%|          | 0/50 [00:00<?, ?it/s]

malloc_trim available: True
malloc_trim available: True
malloc_trim available: True
[Task 6b39e9fe] BEFORE loading - Large objects in memory:
[Task 6b39e9fe] Worker 368054 memory BEFORE loading: 0.20 GB
malloc_trim available: True
malloc_trim available: True
malloc_trim available: True
malloc_trim available: True
malloc_trim available: True
[Task 0aeff1f8] BEFORE loading - Large objects in memory:
[Task 0aeff1f8] Worker 368183 memory BEFORE loading: 0.20 GB
malloc_trim available: True
malloc_trim available: True
malloc_trim available: True
[Task 524421ec] BEFORE loading - Large objects in memory:
[Task 524421ec] Worker 368197 memory BEFORE loading: 0.20 GB
malloc_trim available: True
malloc_trim available: True
[Task 78e5e624] BEFORE loading - Large objects in memory:
[Task 78e5e624] Worker 368121 memory BEFORE loading: 0.20 GB
malloc_trim available: True
[Task 6a18e3c7] BEFORE loading - Large objects in memory:
[Task 6a18e3c7] Worker 368095 memory BEFORE loading: 0.20 GB
[Task 4118049

  6%|▌         | 3/50 [00:08<01:42,  2.18s/it]

[Task 40141203] Worker 368203 memory AFTER delete: 0.21 GB
[Task 4118049d] Worker 368038 memory AFTER delete: 0.21 GB
[Task 7f810eaa] Worker 368058 memory AFTER delete: 0.21 GB
[TASK] run_swirl RETURNED!
[TASK] Processing 1 vortices...
[TASK] run_swirl RETURNED!
[TASK] Processing 6 vortices...


...

[Task 9b0aae7b] Worker 368110 memory AFTER delete: 0.22 GB
[Task be29d4fe] Worker 368054 memory AFTER delete: 0.23 GB


2025-11-19 10:56:53,505 - distributed.worker - ERROR - 
Traceback (most recent call last):
  File "/home/leroquan@eawag.wroot.emp-eaw.ch/miniconda3/envs/swirl_toolbox/lib/python3.11/site-packages/distributed/compatibility.py", line 204, in asyncio_run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/home/leroquan@eawag.wroot.emp-eaw.ch/miniconda3/envs/swirl_toolbox/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/leroquan@eawag.wroot.emp-eaw.ch/miniconda3/envs/swirl_toolbox/lib/python3.11/asyncio/base_events.py", line 641, in run_until_complete
    self.run_forever()
  File "/home/leroquan@eawag.wroot.emp-eaw.ch/miniconda3/envs/swirl_toolbox/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    self._run_once()
  File "/home/leroquan@eawag.wroot.emp-eaw.ch/miniconda3/envs/swirl_toolbox/lib/python3.11/asyncio/base_events.py", line 1898, in _run_o

[Task cc22d3c6] Worker 368067 memory AFTER delete: 0.23 GB
[Task 70140762] Worker 368185 memory AFTER delete: 0.23 GB


2025-11-19 10:56:53,790 - distributed.worker - ERROR - 
Traceback (most recent call last):
  File "/home/leroquan@eawag.wroot.emp-eaw.ch/miniconda3/envs/swirl_toolbox/lib/python3.11/site-packages/distributed/compatibility.py", line 204, in asyncio_run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/home/leroquan@eawag.wroot.emp-eaw.ch/miniconda3/envs/swirl_toolbox/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/leroquan@eawag.wroot.emp-eaw.ch/miniconda3/envs/swirl_toolbox/lib/python3.11/asyncio/base_events.py", line 641, in run_until_complete
    self.run_forever()
  File "/home/leroquan@eawag.wroot.emp-eaw.ch/miniconda3/envs/swirl_toolbox/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    self._run_once()
  File "/home/leroquan@eawag.wroot.emp-eaw.ch/miniconda3/envs/swirl_toolbox/lib/python3.11/asyncio/base_events.py", line 1898, in _run_o

KeyboardInterrupt: 